# NIRCam PSF-matched multiband photometry

In [ ]:
import numpy as np
import os
from glob import glob
from copy import deepcopy
from os.path import expanduser
home = expanduser("~")

import astropy  # version 4.2 is required to write magnitudes to ecsv file
from astropy.io import fits
import astropy.wcs as wcs
from astropy.table import QTable, Table
import astropy.units as u
from astropy.visualization import make_lupton_rgb, SqrtStretch, LogStretch, hist, simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.coordinates import SkyCoord

import photutils
from photutils import Background2D, MedianBackground, detect_sources, deblend_sources, SourceCatalog#, source_properties
from photutils.utils import calc_total_error
print('photutils', photutils.__version__)
from packaging import version
if version.parse(photutils.__version__) < version.parse("1.4.0"):
    print('WARNING photutils not up to date with 1.4; convolution may take forever and will tweak error input command below')

from photutils.psf.matching import resize_psf, create_matching_kernel, CosineBellWindow
from astropy.convolution import convolve, convolve_fft # , Gaussian2DKernel, Tophat2DKernel

from scipy import ndimage

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker
#%matplotlib inline   # non-interactive (easier for notebook scrolling)
%matplotlib notebook
plt.style.use('https://www.stsci.edu/~dcoe/matplotlibrc.txt') # https://matplotlib.org/tutorials/introductory/customizing.html
mpl_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
# Show versions of Python and imported libraries
try:
    import watermark
    %load_ext watermark
    # %watermark -n -v -m -g -iv
    %watermark -iv -v
except ImportError:
    pass

In [ ]:
#image_files_list = glob('../images/*_drz.fits')  
image_files_list = glob('../images/*_sci.fits.gz')
image_files_list = list(np.sort(image_files_list))
filters = [image_file.split('_')[-2] for image_file in image_files_list]
image_files = {}
wavelengths = np.array([int(filt[1:4]) / 100 for filt in filters]) * u.um  # e.g., F115W = 1.15 microns
for i, filt in enumerate(filters):
    image_files[filt] = image_files_list[i]
    print(filt, image_files[filt])

field = os.path.basename(image_files[filt]).split('_')[0]
field

In [ ]:
# Data Error arrays
data_error_files = {}
for filt in filters:
    #data_error_files[filt] = image_files[filt].replace('_drz.fits', '_err.fits')
    data_error_files[filt] = image_files[filt].replace('_sci.fits', '_err.fits')
    print(filt, data_error_files[filt])

## Load detection image: F200W

In [ ]:
idata = 1        # extension of HDU where data lives
idata_error = 2  # extension of HDU where data_error lives

detection_filter = filt = 'F200W'.lower()
infile = image_files[filt]
hdu = fits.open(infile)
data = hdu[idata].data
imwcs = wcs.WCS(hdu[idata].header, hdu)
data_error = fits.open(data_error_files[filt])[idata_error].data

## Report image size and field of view

In [ ]:
ny, nx = data.shape
# image_pixel_scale = np.abs(hdu[0].header['CD1_1']) * 3600
image_pixel_scale = wcs.utils.proj_plane_pixel_scales(imwcs)[0] 
image_pixel_scale *= imwcs.wcs.cunit[0].to('arcsec')
outline = '%d x %d pixels' % (ny, nx)
outline += ' = %g" x %g"' % (ny * image_pixel_scale, nx * image_pixel_scale)
outline += ' (%.2f" / pixel)' % image_pixel_scale
print(outline)

In [ ]:
fig = plt.figure(figsize=(9.5, 8))

ax = fig.add_subplot(2, 1, 1, projection=imwcs)  # , sharex=True, sharey=True
norm = ImageNormalize(stretch=SqrtStretch(), vmin=0, vmax=0.5)
plt.imshow(data, origin='lower', norm=norm, interpolation='none', cmap='Greys')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')

ax = fig.add_subplot(2, 1, 2, projection=imwcs)
norm = ImageNormalize(stretch=SqrtStretch(), vmin=0, vmax=0.5)
plt.imshow(data_error, origin='lower', norm=norm, interpolation='none', cmap='Greys')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')

## Convert measured fluxes (data units) to magnitudes

https://docs.astropy.org/en/stable/units/

https://docs.astropy.org/en/stable/units/equivalencies.html#photometric-zero-point-equivalency

https://docs.astropy.org/en/stable/units/logarithmic_units.html#logarithmic-units

JWST data units: MJy / sr  
Mega Janskys (MJy); mag_AB = 8.9 - 2.5 log10(flux_Jy) = 31.4 - 2.5 log10(flux_nJy)  
steradian (sr) is a unit of solid angle: sphere = 4 pi sr = 4 pi (180 deg / pi)^2 = 41253 deg^2 = 2.2e16 arcsec^2

In [ ]:
# not detected: mag =  99; magerr = 1-sigma upper limit assuming zero flux
# not observed: mag = -99; magerr = 0
def fluxes2mags(flux, fluxerr):
    nondet = flux < 0  # Non-detection if flux is negative
    unobs = (fluxerr <= 0) + (fluxerr == np.inf)  # Unobserved if flux uncertainty is negative or infinity

    mag = flux.to(u.ABmag)
    magupperlimit = fluxerr.to(u.ABmag) # 1-sigma upper limit if flux=0

    mag = np.where(nondet, 99 * u.ABmag, mag)
    mag = np.where(unobs, -99 * u.ABmag, mag)

    magerr = 2.5 * np.log10(1 + fluxerr/flux) 
    magerr = magerr.value * u.ABmag

    magerr = np.where(nondet, magupperlimit, magerr)
    magerr = np.where(unobs, 0 * u.ABmag, magerr)
    
    return mag, magerr

Note magnitude uncertainties for detections should probably be u.mag instead of u.ABmag  
but magnitude uncertainties for non-detections quote u.ABmag upper limits  
They need to be the same, so we go with u.ABmag  

## Detect Sources and Deblend using astropy.photutils (similar to SourceExtractor)
https://photutils.readthedocs.io/en/latest/segmentation.html

In [ ]:
# Define all detection and measurement parameters here so that we do measurements consistently for every image

JWST_flux_units = u.MJy / u.sr

# https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.SourceCatalog.html#photutils.segmentation.SourceCatalog.area
output_properties = 'label xcentroid ycentroid sky_centroid area semimajor_sigma semiminor_sigma'.split()
output_properties += 'fwhm ellipticity orientation gini'.split()
output_properties += 'kron_radius local_background segment_flux segment_fluxerr kron_flux kron_fluxerr'.split()
# columns += 'source_sum source_sum_err kron_flux kron_fluxerr kron_radius local_background'.split()

class Photutils_Catalog:
    def __init__(self, filt, image_file=None, verbose=True, mask_edge_thickness=10):
        self.image_file = image_file or image_files[filt]
        self.hdu = fits.open(self.image_file)
        self.imwcs = wcs.WCS(self.hdu[idata].header, self.hdu)
        self.data       = self.hdu[idata].data  # 'SCI' i2d extension 1
        self.convolved_data = None  # initialize; to be populated later; added in photutils 1.4

        # total error array (i.e., the background-only error plus Poisson noise due to individual sources)
        # https://photutils.readthedocs.io/en/stable/segmentation.html#photometric-errors        
        #self.data_error = self.hdu[2].data  # 'ERR' i2d extension 2
        self.data_error_file = data_error_files[filt]
        self.data_error = fits.open(self.data_error_file)[idata_error].data
        self.data_mask = np.isnan(self.data_error) # | (model.wht == 0)
        # Remove edge detections: Grow the mask by 10 pixels
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.binary_dilation.html
        self.data_mask = ndimage.binary_dilation(self.data_mask, iterations=mask_edge_thickness)

        # image_pixel_scale = np.abs(hdu[0].header['CD1_1']) * 3600
        self.pixel_scale = wcs.utils.proj_plane_pixel_scales(self.imwcs)[0] 
        self.pixel_scale *= imwcs.wcs.cunit[0].to('arcsec')
        self.flux_units = JWST_flux_units * (self.pixel_scale * u.arcsec)**2
        #self.zeropoint = self.hdu[0].header['ABMAG'] * u.ABmag 
        #self.zeropoint = 31.4 * u.ABmag # no zeropoint (ABMAG) in the header, add manually, 31.4 for nJy
        if verbose:
            print(self.image_file)
            ny, nx = self.data.shape
            outline = '%d x %d pixels' % (ny, nx)
            outline += ' = %g" x %g"' % (ny * self.pixel_scale, nx * self.pixel_scale)
            outline += ' (%.2f" / pixel)' % self.pixel_scale
            print(outline)
            #print(filt, '  zeropoint =', self.zeropoint)
            #print(self.weight_file)

    def measure_background_map(self, bkg_size=50, filter_size=3, verbose=True):
        # Calculate sigma-clipped background in cells of 50x50 pixels, then median filter over 3x3 cells
        # For best results, the image should span an integer number of cells in both dimensions (e.g., 1000=20x50 pixels)
        # https://photutils.readthedocs.io/en/stable/background.html
        self.background_map = Background2D(self.data, bkg_size, filter_size=filter_size)

    def smooth_data(self, smooth_fwhm=2, kernel_size=5):
        # convolve data with Gaussian 
        # convolved_data used for source detection and to calculate source centroids and morphological properties
        smooth_sigma = smooth_fwhm * gaussian_fwhm_to_sigma
        self.smooth_kernel = Gaussian2DKernel(smooth_sigma, x_size=kernel_size, y_size=kernel_size)
        self.smooth_kernel.normalize()
        self.convolved_data = convolve(self.data, self.smooth_kernel)
        
    def run_detect_sources(self, nsigma, npixels, smooth_fwhm=2, kernel_size=5, 
                           deblend_levels=32, deblend_contrast=0.001, verbose=True):

        # Set detection threshold map as nsigma times RMS above background pedestal
        detection_threshold = (nsigma * self.background_map.background_rms) + self.background_map.background

        # Before detection, convolve data with Gaussian
        self.smooth_data(smooth_fwhm, kernel_size)

        # Detect sources with npixels connected pixels at/above threshold in data smoothed by kernel
        # https://photutils.readthedocs.io/en/stable/segmentation.html
        self.segm_detect = detect_sources(self.data, detection_threshold, npixels=npixels, kernel=self.smooth_kernel)

        # Deblend: separate connected/overlapping sources
        # https://photutils.readthedocs.io/en/stable/segmentation.html#source-deblending
        self.segm_deblend = deblend_sources(self.data, self.segm_detect, npixels=npixels, kernel=self.smooth_kernel,
                                            nlevels=deblend_levels, contrast=deblend_contrast)
        if verbose:
            output = 'Cataloged %d objects' % self.segm_deblend.nlabels
            output += ', deblended from %d detections' % self.segm_detect.nlabels
            median_threshold = (nsigma * self.background_map.background_rms_median) \
                + self.background_map.background_median
            output += ' with %d pixels above %g-sigma threshold' % (npixels, nsigma)
            # Background outputs equivalent to those reported by SourceExtractor
            output += '\n'
            output += 'Background median %g' % self.background_map.background_median
            output += ', RMS %g' % self.background_map.background_rms_median
            output += ', threshold median %g' % median_threshold
            print(output)

    def measure_source_properties(self, local_background_width=24, properties=output_properties):
        if version.parse(photutils.__version__) >= version.parse("1.4.0"):
            self.catalog = SourceCatalog(self.data-self.background_map.background, self.segm_deblend,
                                         convolved_data=self.convolved_data,  # photutils 1.4
                                         error=self.data_error, mask=self.data_mask,
                                         background=self.background_map.background, wcs=self.imwcs,
                                         localbkg_width=local_background_width)
        else:  # use filter_kernel instead of convolved_data
            self.catalog = SourceCatalog(self.data-self.background_map.background, self.segm_deblend, 
                                         kernel=self.smooth_kernel,  # photutils < 1.4 
                                         error=self.data_error, mask=self.data_mask,
                                         background=self.background_map.background, wcs=self.imwcs,
                                         localbkg_width=local_background_width)


        self.catalog_table = self.catalog.to_table(columns=properties)  # properties: quantities to keep
        
        # Convert fluxes to nJy units and to AB magnitudes
        for aperture in ['segment', 'kron']:
            flux    = self.catalog_table[aperture+'_flux']    * self.flux_units.to(u.nJy)
            fluxerr = self.catalog_table[aperture+'_fluxerr'] * self.flux_units.to(u.nJy)
            mag, magerr = fluxes2mags(flux, fluxerr)
            
            self.catalog_table[aperture+'_flux']    = flux
            self.catalog_table[aperture+'_fluxerr'] = fluxerr
            self.catalog_table[aperture+'_mag']     = mag
            self.catalog_table[aperture+'_magerr']  = magerr            

In [ ]:
detection_catalog = Photutils_Catalog(detection_filter)
detection_catalog.measure_background_map()
detection_catalog.run_detect_sources(nsigma=3, npixels=5)

In [ ]:
detection_catalog = Photutils_Catalog(detection_filter)
detection_catalog.measure_background_map()
detection_catalog.run_detect_sources(nsigma=3, npixels=5)

In [ ]:
# Remove masked segments (labels)
# https://photutils.readthedocs.io/en/latest/api/photutils.segmentation.SegmentationImage.html#photutils.segmentation.SegmentationImage.remove_masked_labels
detection_catalog.segm_deblend.remove_masked_labels(detection_catalog.data_mask)

In [ ]:
# Save segmentation map of detected objects
segm_hdu = fits.PrimaryHDU(detection_catalog.segm_deblend.data.astype(np.uint32), header=imwcs.to_header())
outroot = field + '_' + detection_filter
segm_hdu.writeto(outroot+'_detections_segm.fits', overwrite=True)

In [ ]:
detection_catalog.measure_source_properties()

In [ ]:
#detection_catalog.catalog_table  # show contents

In [ ]:
# Tweak catalog
source_table = detection_catalog.catalog_table
source_table.rename_column('label', 'id')
source_table.rename_column('semimajor_sigma', 'a')
source_table.rename_column('semiminor_sigma', 'b')
source_table.rename_column('xcentroid', 'x')
source_table.rename_column('ycentroid', 'y')

# Replace sky_centroid with ra, dec
source_table['ra']  = source_table['sky_centroid'].ra.degree  * u.degree
source_table['dec'] = source_table['sky_centroid'].dec.degree * u.degree

columns = list(source_table.columns)
columns = columns[:3] + ['ra', 'dec'] + columns[4:-2]

source_table = source_table[columns]

In [ ]:
# If interested, view / save output, but photometry in other filters will be added soon!
if 0:
    source_table.write(outroot+'_detections.ecsv', overwrite=True)
    source_table.write(outroot+'_detections.cat', format='ascii.fixed_width_two_line', delimiter=' ', overwrite=True)
    #source_table  # show contents
    outroot+'_detections.cat'

## Show detections (segmentation map)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9.5,5), sharex=True, sharey=True)

norm = simple_norm(detection_catalog.data, 'sqrt', min_percent=0.1, max_percent=99.9)
ax[0].imshow(detection_catalog.data, origin='lower', interpolation='none', norm=norm, cmap='Greys_r' )
#ax[0].plot(source_table['x'], source_table['y'], 'mo', mfc='None')

cmap = detection_catalog.segm_deblend.make_cmap(seed=123)
ax[1].imshow(detection_catalog.segm_deblend, origin='lower', interpolation='none', cmap=cmap)

for ix in range(2):
    ax[ix].axis('off')

## Multiband photometry using isophotal apertures defined in detection image
(Similar to running SourceExtractor in double-image mode)  
(No PSF corrections just yet)

In [ ]:
for filt in filters:
    filter_catalog = Photutils_Catalog(filt)
    filter_catalog.measure_background_map()
    filter_catalog.segm_deblend = detection_catalog.segm_deblend
    filter_catalog.measure_source_properties()
    
    source_table[filt+'_flux']    = filter_catalog.catalog_table['segment_flux']
    source_table[filt+'_fluxerr'] = filter_catalog.catalog_table['segment_fluxerr']

    source_table[filt+'_mag']     = filter_catalog.catalog_table['segment_flux']
    source_table[filt+'_magerr']  = filter_catalog.catalog_table['segment_magerr']

## Aperture corrections: isophotal to total (Kron aperture) fluxes

In [ ]:
total_flux_table = deepcopy(source_table) # copy to new table, which will include total magnitude corrections

reference_flux_auto = total_flux_table['kron_flux']    # Kron total flux estimate
reference_flux_iso  = total_flux_table['segment_flux'] # flux in isophotal aperture defined by detection segment
kron_flux_corrections = reference_flux_auto / reference_flux_iso
total_flux_table['total_flux_cor'] = kron_flux_corrections

for filt in filters:
    total_flux_table[filt+'_flux']    *= kron_flux_corrections
    total_flux_table[filt+'_fluxerr'] *= kron_flux_corrections
    #total_flux_table[filt+'_mag'] = total_flux_table[filt+'_flux'].to(u.ABmag)  # doesn't handle non-detections
    total_flux_table[filt+'_mag'] = fluxes2mags(total_flux_table[filt+'_flux'], total_flux_table[filt+'_fluxerr'])[0]
    # magnitude uncertainty magerr stays the same

## Reformat output catalog for readability (optional)

In [ ]:
isophotal_table = deepcopy(total_flux_table) # copy to new table, which will include PSF-corrections

old_columns = list(isophotal_table.columns)

# Reorder columns
i = old_columns.index('segment_flux')
j = old_columns.index(filters[0]+'_flux')
columns = old_columns[:i] # detection catalog (except source_sum & kron_flux)
columns += old_columns[-1:]  # total_flux_cor
columns += old_columns[j:-1] # photometry in all filters
        
isophotal_table = isophotal_table[columns]

for column in columns:
    isophotal_table[column].info.format = '.4f'

isophotal_table['ra'].info.format  = '11.7f'
isophotal_table['dec'].info.format = ' 11.7f'

isophotal_table['id'].info.format = 'd'
isophotal_table['area'].info.format = '.0f'  # 'd' raises error : incompatible with units (pix2)

## Isophotal Photometry without PSF corrections complete

We recommend proceeding with PSF corrections to photometry in the Long Wavelength Channel. But if you are interested, you may save the catalog now.

In [ ]:
if 0:
    isophotal_table.write(field+'_isophotal_photometry.ecsv', overwrite=True)
    isophotal_table.write(field+'_isophotal_photometry.cat', format='ascii.fixed_width_two_line', delimiter=' ', overwrite=True)
    #isophotal_table

# PSF magnitude corrections

Color corrections are perfomed as described here:
https://www.stsci.edu/~dcoe/ColorPro/color

Note this is different from one common approach, which is to degrade every image to the broadest PSF.

Photometry is only corrected in Long Wavelength images > 2.4 microns. The F200W detection image is convolved (blurred) to the PSF of each Long Wavelength image. Then we correct colors based on the magnitudes lost in that aperture:

* PSF_magnitude_corrections = detection_image_magnitudes - blurred_detection_image_magnitudes

In practice, we actually correct the fluxes:

* PSF_flux_corrections = detection_image_fluxes / blurred_detection_image_fluxes

## Load PSFs

NIRCam PSF files available via:  
https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/nircam_photometry/NIRCam_PSFs/

...were extracted from tar files available on JDox:  
https://jwst-docs.stsci.edu/near-infrared-camera/nircam-predicted-performance/nircam-point-spread-functions#NIRCamPointSpreadFunctions-SimulatedNIRCamPSFs  
PSFs_SW_filters (short wavelength channel): https://stsci.box.com/s/s2lepxr9086gq4sogr3kwftp54n1c5vl  
PSFs_LW_filters (long wavelength channel): https://stsci.box.com/s/gzl7blxb1k3p4n66gs7jvt7xorfrotyb  

Each FITS file contains:  
– hdu[0]: a 4x oversampled PSF  
– hdu[1]: PSF at detector pixel scale (0.031" and 0.063" in the short and long wavelength channels, respectively)  
This notebook will use the latter: PSF at detector pixel scale. We find no advantage to the former for this notebook.

In [ ]:
PSF_inputs = {}
PSF_images = {}

detector_pixel_scales = {'SW': 0.031, 'LW': 0.063}

#input_file_url = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/nircam_photometry/'
#PSF_directory = os.path.join(input_file_url, 'NIRCam_PSFs')
# missing F480M

PSF_directory1 = os.path.join(home, 'NIRCam/photometry/PSFmatched/')
ny_resize = 0  # make them all the same size: the size of the first one measured
# Need cropped to same size to calculate convolution kernels

for i, filt in enumerate(filters):
    lam = wavelengths[i]
    if lam < 2.4 * u.um:
        channel = 'SW'  # Short Wavelength Channel < 2.4 microns
    else:
        channel = 'LW'  # Long Wavelength Channel > 2.4 microns
    
    PSF_directory = os.path.join(PSF_directory1, 'NIRCam_PSFs_'+channel)

    # Load PSF
    PSF_file = 'PSF_%scen_G5V_fov299px_ISIM41.fits' % filt
    # PSF_file = os.path.join('NIRCam_PSFs_' + channel, PSF_file)
    PSF_file = os.path.join(PSF_directory, PSF_file)

    print(PSF_file)
    PSF_hdu = fits.open(PSF_file)
    PSF_inputs[filt] = data = PSF_hdu[1].data  # extension [1] is at pixel scale (not oversampled)
    ny, nx = data.shape
    
    # Resize from detector pixel scale to image pixel scale (here 0.02" / pix)
    detector_pixel_scale = detector_pixel_scales[channel]
    if not ny_resize:  # calculate once, then use for all PSFs
        ny_resize = ny * detector_pixel_scale / image_pixel_scale  # Assume square PSF (ny = nx)
        ny_resize = np.round(ny_resize)
        ny_resize = int((ny_resize // 2) * 2 + 1)  # Make it an odd number of pixels to ensure PSF is centered
        
    PSF_pixel_scale = ny_resize / ny * image_pixel_scale    
    PSF_image = resize_psf(PSF_inputs[filt], PSF_pixel_scale, image_pixel_scale)  # Resize PSF here
    r = (ny_resize - ny) // 2
    if r > 0:  # new size bigger than original
        PSF_image = PSF_image[r:-r, r:-r]  # Trim to same size as input PSFs
    PSF_images[filt] = PSF_image
    # Note PSF is no longer normalized but will be later in convolution step
    print(filt, ny, ny_resize, PSF_image.shape, PSF_images[filt].shape, PSF_pixel_scale, np.sum(PSF_images[filt]))

In [ ]:
# Show PSFs (optional)

fig, ax = plt.subplots(1, len(filters), figsize=(9.5, 1.5), sharex=True, sharey=True)

r = 15  # PSF will be shown out to radius r (pixels)
for i, filt in enumerate(filters):
    data = PSF_images[filt]
    ny, nx = data.shape
    yc = ny // 2
    xc = nx // 2
    stamp = data[yc-r:yc+r+1, xc-r:xc+r+1]
    norm = ImageNormalize(stretch=LogStretch())  # scale each filter individually
    ax[i].imshow(stamp, cmap='Greys_r', norm=norm, origin='lower')
    ax[i].set_title(filt.upper())
    ax[i].axis('off')

# PSF Matching

https://photutils.readthedocs.io/en/stable/psf_matching.html

## Determine PSF convolution kernels

In [ ]:
PSF_kernels = {}
reference_filter = 'f200w'
reference_PSF = PSF_images[reference_filter]
i_reference = filters.index(reference_filter)
window = CosineBellWindow(alpha=0.35)
for filt in filters[i_reference+1:]:  # only for longer wavelength filters
    PSF_kernels[filt] = create_matching_kernel(reference_PSF, PSF_images[filt], window=window)

In [ ]:
reference_image_hdu = fits.open(image_files[reference_filter])
reference_image_data = reference_image_hdu[idata].data[:]

for output_filter in filters[i_reference+1:]:
    output_image = '%s_convolved_%s_to_%s.fits' % (field, reference_filter, output_filter)
    if os.path.exists(output_image):
        print(output_image, 'EXISTS')
    else:
        print(output_filter + '...')
        PSF_kernel = PSF_kernels[output_filter][yc-r:yc+r+1, xc-r:xc+r+1]
        # convolve_fft may be faster than convolve for larger images / kernels
        convolved_image = convolve_fft(reference_image_data, PSF_kernel, normalize_kernel=True)
        reference_image_hdu[idata].data = convolved_image
        print('SAVING %s' % output_image)
        reference_image_hdu = reference_image_hdu[:3]  # only save extensions 0 (header) and 1 ('SCI') and 2 ('ERR')
        # 'ERR' isn't blurred, but keep it to get this file through the analysis
        reference_image_hdu.writeto(output_image)

## Multiband photometry in convolved images

In [ ]:
# Measure and save the fluxes in each blurry image
blurry_catalog = QTable()

for blurry_filter in filters[i_reference+1:]:
    image_file = '%s_convolved_%s_to_%s.fits' % (field, reference_filter, blurry_filter)
    filter_catalog = Photutils_Catalog(blurry_filter, image_file=image_file)
    filter_catalog.measure_background_map()

    # Measure photometry in this filter for objects detected in detected image
    # segmentation map will define isophotal apertures
    filter_catalog.segm_deblend = detection_catalog.segm_deblend
    filter_catalog.measure_source_properties()

    # Convert measured fluxes to fluxes in nJy
    filter_table = filter_catalog.catalog.to_table()
    blurry_catalog[blurry_filter+'_flux'] = filter_catalog.catalog_table['segment_flux']

## PSF magnitude corrections

https://www.stsci.edu/~dcoe/ColorPro/color

In [ ]:
PSF_corrected_table = deepcopy(isophotal_table)

reference_fluxes = PSF_corrected_table[reference_filter+'_flux']  # det_flux_auto

for filt in filters[i_reference+1:]:
    # Convert isophotal fluxes to total fluxes
    blurry_total_fluxes = blurry_catalog[filt+'_flux'] * kron_flux_corrections
    PSF_flux_corrections = reference_fluxes / blurry_total_fluxes
    PSF_corrected_table[filt+'_flux']    *= PSF_flux_corrections
    PSF_corrected_table[filt+'_fluxerr'] *= PSF_flux_corrections
    # PSF_corrected_table[filt+'_mag'] = PSF_corrected_fluxes.to(u.ABmag)  # doesn't handle non-detections
    PSF_corrected_table[filt+'_mag'], PSF_corrected_table[filt+'_magerr'] = \
        fluxes2mags(PSF_corrected_table[filt+'_flux'], PSF_corrected_table[filt+'_fluxerr'])
    PSF_corrected_table[filt+'_PSF_flux_cor'] = PSF_flux_corrections    

In [ ]:
# Flux in uJy (should have done that above)
for filt in filters:
    PSF_corrected_table[filt+'_flux']    = PSF_corrected_table[filt+'_flux'].to(u.uJy)
    PSF_corrected_table[filt+'_fluxerr'] = PSF_corrected_table[filt+'_fluxerr'].to(u.uJy)    

In [ ]:
PSF_corrected_table.write(field+'_photometry.ecsv', overwrite=True)
PSF_corrected_table.write(field+'_photometry.cat', format='ascii.fixed_width_two_line', delimiter=' ', overwrite=True)

In [ ]:
PSF_corrected_table.write(field+'_photometry.csv', overwrite=True)  # for EAZY with flux in uJy

In [ ]:
field+'_photometry.cat'

In [ ]:
PSF_corrected_table